This notebook does a very preliminary assessment of the existing tables, and tries to create metadata that is machine and human readable.

In [1]:
import pandas as pd
from tableschema import Table
from goodtables import validate

this file has a sort of description of description of what do we have here. some of it could eventually be moved to the metadata 

In [2]:
overview=pd.read_csv('raw/_Overview_of_available_correspondence_files.csv',
            engine='python',
            )

drop cells and rows that are completely empty

In [20]:
overview=overview.dropna(how='all',axis='columns').dropna(how='all',axis='index')

In [22]:
overview[['Full name','Full name.1']]

,Full name,Full name.1
1,Standard International Trade Classification,Central Product Classification
2,Standard International Trade Classification,Broad Economic Categories
3,Standard International Trade Classification,Combined Nomenclature
4,Standard International Trade Classification,Harmonized System
5,Central Product Classification,Standard International Trade Classification
6,Central Product Classification,FAOSTAT Commodity List
7,Central Product Classification,Classification of Products by Activity
8,United Nations Standard Products and Services ...,Standard International Trade Classification
9,FAOSTAT Commodity List,Central Product Classification
10,Broad Economic Categories,Standard International Trade Classification


it could be a good idea to use use always lowercase.

In [23]:
overview.loc[1]

FROM classification:                                                         SITC
Full name                             Standard International Trade Classification
Author/organization                                                United Nations
TO classification:                                                            CPC
Full name.1                                        Central Product Classification
Author/organization.1                                              United Nations
Type of clasifications                                                   products
Status of the correspondance file                          Completed and Verified
Comment                                                                       NaN
Name: 1, dtype: object

## Exiobase2 nace2008

it is interpreting the last column as if there was something but it is empty. probably it would be necessary to know the version of exiobase 

In [73]:
ex_to_nace_df=pd.read_csv('raw/exiobase2_to_NACE2008_v2.csv')
ex_to_nace_df.head(3)

,IndustryTypeCode,IndustryTypeName,NACE Code A,A,Unnamed: 4
0,i01.a,Cultivation of paddy rice,1.10,Growing of non-perennial crops,NaN
1,i01.a,Cultivation of paddy rice,1.12,Growing of rice,NaN
2,i01.b,Cultivation of wheat,1.11,"Growing of cereals (except rice), leguminous c...",NaN


there is a 4th column without name which is mostly empty.

In [20]:
ex_to_nace_df[ex_to_nace_df['Unnamed: 4'].notna()]

,IndustryTypeCode,IndustryTypeName,NACE Code A,A,Unnamed: 4
68,i14.1,Quarrying of stone,8.11,"Quarrying of ornamental and building stone, li...",08.9 Mining and quarrying n.e.c. MINUS 08.92 E...
112,i23.3,Processing of nuclear fuel,20.13,Manufacture of other inorganic basic chemicals,Enrichment of uranium
113,i23.3,Processing of nuclear fuel,21.20,Manufacture of pharmaceutical preparations,Radioactive in-vivo diagnostic substances
114,i23.3,Processing of nuclear fuel,24.46,Processing of nuclear fuel,Except enrichment of uranium.
115,i23.3,Processing of nuclear fuel,38.12,Collection of hazardous waste,Collection and treatment of radioactive nuclea...
116,i23.3,Processing of nuclear fuel,38.22,Treatment and disposal of hazardous waste,"Treatment, disposal and storage of radioactive..."


lets work with the assumption that we erase that almost empty column.

In [24]:
ex_to_nace_df=ex_to_nace_df.drop('Unnamed: 4',axis=1)

In [30]:
ex_to_nace_df.to_csv('final_tables/exiobase2_to_NACE2008_v2.csv',index=None)

## create a frictionless data descriptor (metadata)

example here : https://github.com/frictionlessdata/tableschema-py

In [27]:
ex_to_nace = Table('final_tables/exiobase2_to_NACE2008_v2.csv')

In [31]:
ex_to_nace.read(keyed=True)[0]

{'IndustryTypeCode': 'i01.a',
 'IndustryTypeName': 'Cultivation of paddy rice',
 'NACE Code A': '1.1',
 'A': 'Growing of non-perennial crops'}

infer data types

In [32]:
ex_to_nace.infer()

{'fields': [{'name': 'IndustryTypeCode',
   'type': 'string',
   'format': 'default'},
  {'name': 'IndustryTypeName', 'type': 'string', 'format': 'default'},
  {'name': 'NACE Code A', 'type': 'number', 'format': 'default'},
  {'name': 'A', 'type': 'string', 'format': 'default'}],
 'missingValues': ['']}

In [33]:
ex_to_nace.schema.descriptor

{'fields': [{'name': 'IndustryTypeCode',
   'type': 'string',
   'format': 'default'},
  {'name': 'IndustryTypeName', 'type': 'string', 'format': 'default'},
  {'name': 'NACE Code A', 'type': 'number', 'format': 'default'},
  {'name': 'A', 'type': 'string', 'format': 'default'}],
 'missingValues': ['']}

In [34]:
ex_to_nace.read(keyed=True)

CastError: Table headers don't match schema field names

alternatively use the web interface : http://try.goodtables.io/

In [37]:
report=validate('final_tables/exiobase2_to_NACE2008_v2.csv')

In [49]:
for error in report['tables'][0]['errors']:
    print(error['code'],error['row-number'])

duplicate-row 38
duplicate-row 41
duplicate-row 49
duplicate-row 108
duplicate-row 165
duplicate-row 184
duplicate-row 186
duplicate-row 191
duplicate-row 193
duplicate-row 195
duplicate-row 243
duplicate-row 245


there are indeed 12 duplicated rows.

In [52]:
ex_to_nace_df.duplicated().sum()

12

In [55]:
ex_to_nace_df[ex_to_nace_df.duplicated(keep=False)]

,IndustryTypeCode,IndustryTypeName,NACE Code A,A
35,i01.l,Meat animals nec,1.70,"Hunting, trapping and related service activities"
36,i01.l,Meat animals nec,1.70,"Hunting, trapping and related service activities"
38,i01.m,Animal products nec,10.10,Processing and preserving of meat and producti...
39,i01.m,Animal products nec,10.10,Processing and preserving of meat and producti...
46,i01.w.1,"Manure treatment (conventional), storage and l...",NaN,NaN
47,i01.w.1,"Manure treatment (conventional), storage and l...",NaN,NaN
105,i21.2,Paper,17.12,Manufacture of paper and paperboard
106,i21.2,Paper,17.12,Manufacture of paper and paperboard
162,i29,Manufacture of machinery and equipment n.e.c.,28.30,Manufacture of agricultural and forestry machi...
163,i29,Manufacture of machinery and equipment n.e.c.,28.30,Manufacture of agricultural and forestry machi...


In [57]:
ex_to_nace_df=ex_to_nace_df.drop_duplicates()

In [58]:
ex_to_nace_df.to_csv('final_tables/exiobase2_to_NACE2008_v2.csv',index=None)

In [59]:
report=validate('final_tables/exiobase2_to_NACE2008_v2.csv')

In [64]:
if (report['error-count']==0): print('hurray!')

hurray!


now that it is error free we can create the description

we can add some fields to the descriptor:

In [71]:
ex_to_nace.schema.descriptor['description']=\
'table linking exiobase v2 with NACE2008 (Statistical Classification of Economic Activities in the European Community)'

In [72]:
ex_to_nace.schema.save('final_tables/exiobase2_to_NACE2008_v2.json')
#ex_to_nace.save('data.csv')